In [15]:
# Importing packages
import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq
import os

In [36]:
# Reading text file
path = "C:\\Users\\Lenovo\\Desktop\\Spring 2021\\AMS 380\\Project\\TheAdventuresOfSherlockHolmes.txt"
text = open(path, encoding="utf8").read().lower()
print('Length of text:', len(text))

Length of text: 562172


In [39]:
# Converting text file to a list, while removing all spaces and puntuation
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [41]:
# Creating a dictionary of words (after removing duplicates) as key, and their index position as the value.
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [42]:
# Feature Engineering
WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['the', 'adventures', 'of', 'sherlock', 'holmes']
by


In [44]:
# One-Hot Encoding (using True and Falses)
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

print(X[0][0])

[False False False ... False False False]


In [47]:
# LSTM type RNN model
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [48]:
# Creating Training and Testing Data (80% vs. 20%) and them splitting them into hierarchical data (.h5) 

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history

model.save('keras_next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))
model = load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

Epoch 1/2
787/787 [==============================] - 190s 241ms/step - loss: 6.0287 - accuracy: 0.1048 - val_loss: 5.6941 - val_accuracy: 0.1287
Epoch 2/2
787/787 [==============================] - 164s 209ms/step - loss: 5.7682 - accuracy: 0.1468 - val_loss: 5.9628 - val_accuracy: 0.1395


In [50]:
# Creating function to predict the testing data
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x
prepare_input("It is not a lack".lower())

it
is
not
a
lack


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [51]:
# Creating function to find the 3 most used words after the given word
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [52]:
# The final prediction models
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

In [73]:
# Trying out the RNN model
q = "In my dealings with Sherlock Holmes"
print("correct sentence: ", q)
print("\nSequence of words:")
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
l = predict_completions(seq, 3)
print("\nNext Possible Words: ", end="")
for i in range(len(l)):
    if i == len(l)-1:
        print(l[i], end=".")
    else:
        print(l[i], end=", ")

correct sentence:  In my dealings with Sherlock Holmes

Sequence of words:
in
my
dealings
with
sherlock

Next Possible Words: holmes, he, that.